# 8 数据规整：连接、联合与重塑
重点关注数据联合、连接以及重排

## 8.1 分层索引

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 双层索引的Series
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.421656
   2   -0.125544
   3   -0.201874
b  1    0.484224
   3   -0.674129
c  1    0.466865
   2   -1.578992
d  2   -0.686441
   3   -0.369766
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
# 使用分层索引可以简洁地选择出数据的子集
data['b']

1    0.484224
3   -0.674129
dtype: float64

In [5]:
data['b':'c']

b  1    0.484224
   3   -0.674129
c  1    0.466865
   2   -1.578992
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1    0.484224
   3   -0.674129
d  2   -0.686441
   3   -0.369766
dtype: float64

In [8]:
# 内部层级中也可以选择，比如选择内层中索引为2的
data.loc[:, 2]

a   -0.125544
c   -1.578992
d   -0.686441
dtype: float64

In [11]:
# 重塑数据
data.unstack()

,1,2,3
a,-0.421656,-0.125544,-0.201874
b,0.484224,NaN,-0.674129
c,0.466865,-1.578992,NaN
d,NaN,-0.686441,-0.369766


In [15]:
# 在DataFrame中，每个轴都可以有分层索引
df = pd.DataFrame(np.arange(12).reshape(4, 3),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'California'], ['Green', 'Red', 'Green']])
df.index.names = ['key1', 'key2']
df.columns.names = ['State', 'Color']
df

State      Ohio     California
Color     Green Red      Green
key1 key2                     
a    1        0   1          2
     2        3   4          5
b    1        6   7          8
     2        9  10         11

In [16]:
# 筛选出数据
df['Ohio']

Color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [18]:
df.loc['a']

State  Ohio     California
Color Green Red      Green
key2                      
1         0   1          2
2         3   4          5

In [28]:
from pandas import MultiIndex

columns = MultiIndex.from_arrays([['Ohio', 'Ohio', 'California'], ['Green', 'Red', 'Green']],
                                 names=['State', 'Color'])
indexes = MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                                 names=['key1', 'key2'])

In [29]:
df2 = pd.DataFrame(np.arange(12).reshape(4, 3),
                   columns=columns,
                   index=indexes)

In [30]:
df2

State      Ohio     California
Color     Green Red      Green
key1 key2                     
a    1        0   1          2
     2        3   4          5
b    1        6   7          8
     2        9  10         11

### 8.1.1 重排序和层级排序

In [32]:
# 重新排列轴上的层级顺序,
# swaplevel接收两个层级序号或层级名称，返回一个层级变更的新对象
df2.swaplevel('key1', 'key2')

State      Ohio     California
Color     Green Red      Green
key2 key1                     
1    a        0   1          2
2    a        3   4          5
1    b        6   7          8
2    b        9  10         11

In [36]:
# level=0表示第一层(key1)，level=1表示第二层(key2)
# 如果索引按照字典顺序从最外层开始排序，那么数据选择性能会更好
df2.sort_index(level=0, ascending=False)

State      Ohio     California
Color     Green Red      Green
key1 key2                     
b    2        9  10         11
     1        6   7          8
a    2        3   4          5
     1        0   1          2

### 8.1.2 按层级进行汇总统计
Series和DataFrame上很多描述性和统计行统计有一个level选项，通过level可以指定要在某个特定的轴上进行聚合

In [38]:
df2.groupby(level='key1').sum()

State  Ohio     California
Color Green Red      Green
key1                      
a         3   5          7
b        15  17         19

In [39]:
df2.groupby(level='Color', axis=1).sum()

Color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 使用DataFrame的列进行索引

In [41]:
df3 = pd.DataFrame({'a': range(7),
                    'b': range(7, 0, -1),
                    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                    'd': [0, 1, 2, 0, 1, 2, 3]})
df3

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [42]:
df3.set_index(['c', 'd'])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [43]:
# 默认情况下，设置为index的列会从DataFrame中移除，也可以保留
df3.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [46]:
# reset_index是set_index的反向操作
df4 = df3.set_index(['c', 'd'])
df4.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 联合与合并数据集

In [47]:
# 数据库风格的DataFrame连接
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [48]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [49]:
# 使用pd.merge连接
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [51]:
# 如果列名不同，可以手动指定连接列
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [53]:
# pd.merge默认情况下是内连接，取交集
# 可以指定连接方式 outer为外连接，取并集
pd.merge(df3, df4, how='outer', left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [54]:
# 使用多键合并时，传入一个列名的列表
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'lval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [55]:
# 处理重叠的列名，使用suffixes参数
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 根据索引合并

In [56]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]},
                      index=['a', 'b'])
# 只保留右边的索引
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [59]:
# 只保留左边的索引
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN
